In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

# from wordcloud import WordCloud
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
def text_preprocess(ds):
    for m in range(len(ds)):

        main_words = re.sub('[^a-zA-Z]', ' ', str(ds[m]))                                      # Retain only alphabets
        main_words = (main_words.lower()).split()
        main_words = [w for w in main_words if not w in set(stopwords.words('english'))]  # Remove stopwords

        lem = WordNetLemmatizer()
        main_words = [lem.lemmatize(w) for w in main_words if len(w) > 1]                 # Group different forms of the same word

        main_words = ' '.join(main_words)
        ds[m] = main_words

    return ds

In [ ]:
df = pd.read_csv('drive/MyDrive/Diplomski/cyberbullying_tweets.csv')
X = df['tweet_text'].values
y = df['cyberbullying_type'].values

In [ ]:
X = text_preprocess(X.copy())
# y = text_preprocess(y.copy())

In [ ]:
df['preproceesed_input'] = X
df.to_csv('drive/MyDrive/Diplomski/cyberbullying_tweets_2.csv')

In [ ]:
df = pd.read_csv('drive/MyDrive/Diplomski/cyberbullying_tweets_2.csv')
df.dropna(inplace=True)
X = df['preproceesed_input'].values
y = df['cyberbullying_type'].values

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
print(len(y))
encoded_y = label_encoder.fit_transform(y)
print(len(encoded_y))

47652
47652


In [ ]:
#**************** TF - IDF ******************

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
td = TfidfVectorizer(max_features=3000)
X = td.fit_transform(X).toarray()
# X_val = td.transform(X_val).toarray()

In [ ]:
#*************** embedding ******************

In [ ]:
!unzip /content/drive/MyDrive/Diplomski/glove*.zip

Archive:  /content/drive/MyDrive/Diplomski/glove.6B.zip
replace glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
def embedding_for_vocab(filepath, word_index,
                        embedding_dim):
    vocab_size = len(word_index) + 1

    # Adding again 1 because of reserved 0 index
    embedding_matrix_vocab = np.zeros((vocab_size,
                                       embedding_dim))

    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix_vocab[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix_vocab

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

In [ ]:
embedding_dim = 300
embedding_matrix_vocab = embedding_for_vocab(
    '/content/drive/MyDrive/Diplomski/glove.6B.300d.txt', tokenizer.word_index,
  embedding_dim)


In [ ]:
def text_to_embeddings(text_data, tokenizer, embedding_matrix):
    sequences = tokenizer.texts_to_sequences(text_data)
    padded_sequences = pad_sequences(sequences, maxlen=100, padding='post')

    embeddings = []
    for seq in padded_sequences:
        seq_embedding = [embedding_matrix[word_idx] for word_idx in seq if word_idx != 0]
        if not seq_embedding:
            seq_embedding = [np.zeros(embedding_dim)]  # Handle empty sequences
        embeddings.append(np.mean(seq_embedding, axis=0))

    return np.vstack(embeddings)

In [ ]:
X = text_to_embeddings(X, tokenizer, embedding_matrix_vocab)

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=123)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=123)

In [ ]:
#encoded y
X_train, X_temp, y_train, y_temp = train_test_split(X, encoded_y, test_size=0.3, random_state=123)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=123)

In [ ]:
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

# classifier = svm.SVC(kernel='linear')
# classifier.fit(X_train, y_train)

classifier = svm.SVC()
parameters = {'C': [0.1, 1, 10, 100, 1000],
              'kernel': ['linear']}
kf = StratifiedKFold(n_splits=10, random_state=123, shuffle=True)
gs_clf = GridSearchCV(classifier, parameters, cv=kf)
gs_clf = gs_clf.fit(X_train, y_train)

print("SVM best parameters: ")
print(gs_clf.best_params_)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
# classifier = RandomForestClassifier()
classifier = RandomForestClassifier(bootstrap=False, criterion='gini', n_estimators= 100, random_state= 123, warm_start= True)
classifier.fit(X_train, y_train)

# parameters = {
#             "n_estimators": [10, 50, 100],
#             "criterion": ["gini", "entropy"],
#             "bootstrap": [False],
#             "warm_start": [True],
#             "random_state": [123]
#             }

# kf = StratifiedKFold(n_splits=10, random_state=123, shuffle=True)
# gs_clf = GridSearchCV(classifier, parameters, cv=kf)
# gs_clf = gs_clf.fit(X_train, y_train)

# print("RF best parameters: ")
# print(gs_clf.best_params_)

# validation_score = gs_clf.score(X_val, y_val)
# print(validation_score)


RandomForestClassifier(bootstrap=False, random_state=123, warm_start=True)

In [ ]:
!sudo pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 2.9 MB/s eta 0:00:00


In [ ]:
import xgboost as xgb
classifier = xgb.XGBClassifier(n_estimators=100, subsample=0.7, colsample_bytree=0.6, random_state=123)
classifier.fit(X_train, y_train)
# import xgboost as xgb
# from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
# param_grid = {
#     'max_depth': [3, 5, 7],
#     'learning_rate': [0.1, 0.01, 0.001],
#     'subsample': [0.5, 0.7, 1]
# }

# # Create the XGBoost model object
# xgb_model = xgb.XGBClassifier()

# # Create the GridSearchCV object
# grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='accuracy')

# # Fit the GridSearchCV object to the training data
# grid_search.fit(X_train, y_train)

# # Print the best set of hyperparameters and the corresponding score
# print("Best set of hyperparameters: ", grid_search.best_params_)
# print("Best score: ", grid_search.best_score_)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn import svm
svm = svm.SVC(kernel='rbf')
classifier = BaggingClassifier(estimator=svm, n_estimators=5, random_state=123)
classifier.fit(X_train, y_train)

BaggingClassifier(estimator=SVC(), n_estimators=5, random_state=123)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, f1_score
y_pred = classifier.predict(X_val)

classification_report = classification_report(y_val, y_pred)

# print('\n Accuracy: ', f1_score(y_val, y_pred))
print('\nClassification Report')
print('======================================================')
print('\n', classification_report)
# print('\n', label_encoder.classes_)


Classification Report

                      precision    recall  f1-score   support

                age       0.98      0.98      0.98      1608
          ethnicity       0.99      0.99      0.99      1563
             gender       0.89      0.84      0.86      1563
  not_cyberbullying       0.56      0.54      0.55      1540
other_cyberbullying       0.60      0.64      0.62      1667
           religion       0.97      0.96      0.97      1637

           accuracy                           0.83      9578
          macro avg       0.83      0.83      0.83      9578
       weighted avg       0.83      0.83      0.83      9578



In [ ]:
from sklearn.metrics import accuracy_score, classification_report, f1_score
y_pred = classifier.predict(X_test)

classification_report = classification_report(y_test, y_pred)

# print('\n Accuracy: ', f1_score(y_val, y_pred))
print('\nClassification Report')
print('======================================================')
print('\n', classification_report)
# print('\n', label_encoder.classes_)


Classification Report

               precision    recall  f1-score   support

           0       0.96      0.98      0.97       813
           1       0.97      0.98      0.97       786
           2       0.91      0.85      0.87       749
           3       0.58      0.54      0.56       785
           4       0.60      0.68      0.64       802
           5       0.96      0.94      0.95       783

    accuracy                           0.83      4718
   macro avg       0.83      0.83      0.83      4718
weighted avg       0.83      0.83      0.83      4718

